In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from plotly.offline import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
!pip install pmdarima
from utils import *

In [2]:
datas = []
for i in range(1,3):
    data = pd.read_csv("../data/funda_train_%d주_수정.csv"%(i))
    data["Date"] = data["Date"]=pd.to_datetime(data["Date"], format="%Y-%m-%d")
    datas.append(data)

### 기간별 데이터셋 리딩

In [5]:
sample = datas[0]["store_id"].sample(5)

In [6]:
def decompostion_plot(sample,dat,id_col,int_col,date_col):
    data = dat.copy()
    data.index = data[date_col]
    for i in sample.tolist():
        try:
            result = seasonal_decompose(np.log1p(data[data[id_col]==i][int_col]))
            fig = result.plot()
            plot_mpl(fig)
        except:
            print(i)

In [7]:
decompostion_plot(sample,datas[0],"store_id","amount","Date")

C:\Users\LeeWonSeok\Anaconda3\envs\django\lib\site-packages\plotly\matplotlylib\mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

C:\Users\LeeWonSeok\Anaconda3\envs\django\lib\site-packages\plotly\matplotlylib\mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

C:\Users\LeeWonSeok\Anaconda3\envs\django\lib\site-packages\plotly\matplotlylib\mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.

C:\Users\LeeWonSeok\Anaconda3\envs\django\lib\site-packages\plotly\matplotlylib\mpltools.py:368: MatplotlibDeprecationWarning:


The is_frame_like function was deprecated in Matplotlib 3.1 and will be removed in 3.3.



## Auto arima 모형 소개
- 최적 parmater를 자동으로 찾아서 arima모델을 fitting 해줌
- 기본적으로 p(AR),d(차분),q(MR) 파라미터를 튜닝
- 계절성분 추가시 SARMIA의 P,Q,R 도 튜닝할 수 있음
- AIC,AICC,BIC,HQIC,OOB 등을 고려하여 다음 값이 가장 작은 모델을 리턴함
- oob 는 Out of bag 방식
## Parameter
- y : 예측할 값(nan,inf 혀용 x)
- exogenous : y값이외의 추가 데이터 사용 예측(2차원 array 형태로 input)
- information_criterion : aic,bic,oob 등 피팅기준
- start_p : ARIMA(p,d,q)의 grid_search p 시작값
- start_q : ARIMA(p,d,q)의 grid_search q 시작값
- max_p : ARIMA(p,d,q)의 grid_search p 최대값
- max_q : ARIMA(p,d,q)의 grid_search q 최대값
- d : ARIMA(p,d,q)의 차분성분(None이면 알아서 하지만 long time 보통 1,2 안에)
- start_P : SARIMA(P,D,Q)의 grid_search p 시작값
- start_Q : SARIMA(P,D,v)의 grid_search q 시작값
- max_P : SARIMA(P,D,Q)의 grid_search p 최대값
- max_Q : SARIMA(P,D,Q)의 grid_search q 최대값
- D : SARIMA(p,D,q)의 차분성분(None이면 알아서 하지만 long time 보통 1,2 안에)
- max_order : ARIMA(p,d,q)의 p와 q의 합 최대를 제한 (default 5, None이면 제약없음)
- m : season parameter , 계절을 얼마나 할건지(ex 4계절 4, 월별 12 등 상황따라)
- seasonal : SARIMA 할건지 정하는것  m==1 일 땐 알아서 False임
- stationary : 정상시계열인지 아닌지 True일 때 d==0 이어야함
- stepwise : 단계적선택법을 적용해 paramter 찾을지 여부 (True 추천)
- n_jobs : 지정한 수 범위내에서 가능한 arima model 다 fitting함
- trend : 추세다항식성분 넣을건지 "c" 상수, "t" 1차항하나 , "ct" 상수랑 1차항둘다
- supress_warnings : arima 모델추정시 발생할수있는  warning 무시할건지 
- error_action : 피팅도중 error 발생시 행동 ("ignore","warn")
- trace : 피팅되는과정  print할지말지  
- random : 랜덤 서칭으로 parameter찾음 (단계적이 false일때 씀)
- n_fits : 랜덤 서칭으로 몇번 iteration할건지
- out_of_sample_size : 피팅할때 데이터꼬리 일부를 빼놓는 방식(검증용) , self.arima_res_.data.endog values에 있음
- scoring : {mae,mse} 기준으로 out_of_sample_size>0일 때 검증함

### arima parmeter tunning plan
1. ARIMA(0,0,0) ~ ARIMA(10,2,10) 까지 
2. SARIMA(0,0,0) ~ SARIMA(5,2,5) 까지
3. stepwise = True
4. seasonal = {True, False}
5. m = {half:2,season:4,month:12}
6. out_sample => {1주 :4, 2주:2, 3주:2, 4주:1}
7. scoring = mae
8. supress_warnings = True
9. error_action = "ignore"
10. trend = ["c","t","ct"]

In [5]:
# 주차별 결과 테이블로 나타내기
# 모델링 경과 MAE가 가장 좋은 모델 쓰기
# 피팅된 모델 결과도 기록하기
total_mean_result = pd.DataFrame(columns=["unit","mae","rmse","mae_log","rmse_log"]) 
for unit,data in enumerate(datas,1):
    total_result = pd.DataFrame(columns=["store_id","mae","rmse","mae_log","rmse_log"])
    store_id_lst = data["store_id"].unique()
    for ids in store_id_lst:
        # id filtering and train test split
        each_store = data[data["store_id"]==ids]
        store_train = each_store[each_store["Date"]<'2019-01-01']
        store_test = each_store[each_store["Date"]>='2019-01-01']
        test_period = len(store_test)
        if test_period == 0:
            print(ids)
            continue
        # gird search modeling
        modeling = time_model(np.log1p(store_train["amount"].values))
        result_model = modeling.auto_arimga_grid()
        MAE_result = []
        y_true = store_test["amount"].values
        y_true_log = np.log1p(y_true)
        # best model select
        for each_model in result_model:
            pred = each_model.predict(n_periods = test_period)
            pred = np.expm1(pred)
            mae = modeling.MAE(y_true,pred)
            MAE_result.append(mae)
        best_model = result_model[np.argmin(MAE_result)]
        # calculate metrics
        best_mae = np.min(MAE_result)
        y_pred_log = best_model.predict(n_periods = test_period)
        y_pred = np.expm1(y_pred_log)
        best_rmse = modeling.RMSE(y_true,y_pred)
        mae_log = modeling.MAE(y_true_log,y_pred_log)
        rmse_log = modeling.RMSE(y_true_log,y_pred_log)
        # save_best_model
        modeling.save_model(best_model,'../model/model_%d_%d.sav'%(unit,ids))
        # append reulst to DataFrame
        mini_result = pd.DataFrame.from_dict([{"store_id" :ids,
                                    "mae":best_mae,
                                    "rmse":best_rmse,
                                     "mae_log":mae_log,
                                      "rmse_log":rmse_log}])
        total_result = pd.concat([total_result,mini_result],axis=0,sort=False)
    # caulcate mean result
    mean_mae = total_result["mae"].mean()
    mean_rmse = total_result["rmse"].mean()
    mean_mae_log = total_result["mae_log"].mean()
    mean_rmse_log = total_result["rmse_log"].mean()
    mini_mean_result = pd.DataFrame.from_dict([{"unit":unit,
                                    "mae":mean_mae,
                                    "rmse":mean_rmse,
                                    "mae_log":mean_mae_log,
                                    "rmse_log":mean_rmse_log}])
    total_mean_result = pd.concat([total_mean_result,mini_mean_result],axis=0,sort=False)
    # result to excel
    total_result.to_excel("../result/each_store_result_%d.xlsx"%(unit))
total_mean_result.to_excel('../result/total_unit_result.xlsx')

Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=160.702, BIC=166.103, Fit time=0.014 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=122.380, BIC=130.482, Fit time=0.013 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=140.332, BIC=148.433, Fit time=0.015 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=160.702, BIC=166.103, Fit time=0.013 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=112.959, BIC=123.761, Fit time=0.043 seconds
Fit ARIMA: order=(3, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=114.125, BIC=127.628, Fit time=0.095 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=110.905, BIC=124.407, Fit time=0.105 seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=108.731, BIC=119.533, Fit time=0.095 seconds
Fit ARIMA: order=(1, 0, 2) seasonal_order=(0, 0, 0, 0); AIC=110.638, BIC=124.140, Fit time=0.109 seconds
Fit ARIMA: order=(0, 0, 2) seasonal_order=(0, 0, 0, 0);

Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=112.271, BIC=123.036, Fit time=0.034 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=113.387, BIC=126.844, Fit time=0.067 seconds
Total fit time: 0.188 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=147.046, BIC=152.429, Fit time=0.008 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=122.288, BIC=130.362, Fit time=0.015 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=110.279, BIC=118.353, Fit time=0.021 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=147.046, BIC=152.429, Fit time=0.008 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=112.271, BIC=123.036, Fit time=0.035 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=112.271, BIC=123.036, Fit time=0.033 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=113.387, BIC=126.844, Fit time=0.066 seconds
Total fit time: 0.187 sec

Fit ARIMA: order=(1, 0, 3) seasonal_order=(0, 0, 0, 0); AIC=137.894, BIC=154.096, Fit time=0.128 seconds
Fit ARIMA: order=(0, 0, 3) seasonal_order=(0, 0, 0, 0); AIC=411.886, BIC=425.388, Fit time=0.150 seconds
Near non-invertible roots for order (0, 0, 3)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.990)
Fit ARIMA: order=(2, 0, 3) seasonal_order=(0, 0, 0, 0); AIC=125.629, BIC=144.532, Fit time=0.146 seconds
Near non-invertible roots for order (2, 0, 3)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.993)
Total fit time: 0.949 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=710.300, BIC=715.701, Fit time=0.011 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=156.757, BIC=164.858, Fit time=0.055 seconds
Near non-invertible roots for order (1, 0, 0)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the borde

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=112.568, BIC=123.333, Fit time=0.032 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=112.410, BIC=123.176, Fit time=0.068 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=114.137, BIC=127.594, Fit time=0.084 seconds
Total fit time: 0.299 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=147.035, BIC=152.418, Fit time=0.017 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=122.264, BIC=130.338, Fit time=0.031 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=110.492, BIC=118.566, Fit time=0.050 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=147.035, BIC=152.418, Fit time=0.017 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=112.568, BIC=123.333, Fit time=0.031 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=112.410, BIC=123.176, Fit time=0.068 seconds
Fit ARIMA: order=(1, 1, 2

Fit ARIMA: order=(1, 0, 2) seasonal_order=(0, 0, 0, 0); AIC=114.392, BIC=130.595, Fit time=0.046 seconds
Fit ARIMA: order=(0, 0, 2) seasonal_order=(0, 0, 0, 0); AIC=119.505, BIC=133.008, Fit time=0.100 seconds
Total fit time: 0.961 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=146.500, BIC=154.601, Fit time=0.036 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=119.387, BIC=130.189, Fit time=0.032 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=131.433, BIC=142.235, Fit time=0.084 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=146.500, BIC=154.601, Fit time=0.036 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=112.254, BIC=125.756, Fit time=0.079 seconds
Fit ARIMA: order=(3, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=113.764, BIC=129.967, Fit time=0.095 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=113.826, BIC=130.029, Fit time=0.141 seconds
Fit ARIMA: order=(1, 0, 1

...

Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=355.621, BIC=368.938, Fit time=0.101 seconds
Total fit time: 0.341 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=362.962, BIC=369.620, Fit time=0.012 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=353.862, BIC=362.740, Fit time=0.029 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=356.139, BIC=365.017, Fit time=0.032 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=362.962, BIC=369.620, Fit time=0.014 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=355.337, BIC=366.435, Fit time=0.048 seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=354.151, BIC=365.249, Fit time=0.105 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=355.621, BIC=368.938, Fit time=0.101 seconds
Total fit time: 0.343 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=362.962, BIC=369.620, Fit time=0.012 sec

Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=354.612, BIC=365.710, Fit time=0.064 seconds
Total fit time: 0.241 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=365.306, BIC=369.745, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=353.666, BIC=360.325, Fit time=0.016 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=356.867, BIC=363.525, Fit time=0.012 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=365.306, BIC=369.745, Fit time=0.003 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=354.866, BIC=363.744, Fit time=0.045 seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=353.158, BIC=362.036, Fit time=0.039 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=354.612, BIC=365.710, Fit time=0.064 seconds
Total fit time: 0.183 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=365.306, BIC=369.745, Fit time=0.003 sec

Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=353.422, BIC=364.446, Fit time=0.048 seconds
Total fit time: 0.124 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=427.359, BIC=431.798, Fit time=0.009 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=371.519, BIC=378.178, Fit time=0.027 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=398.317, BIC=404.976, Fit time=0.016 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=427.359, BIC=431.798, Fit time=0.011 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=368.013, BIC=376.891, Fit time=0.028 seconds
Fit ARIMA: order=(3, 0, 0) seasonal_order=(0, 0, 0, 0); AIC=366.232, BIC=377.330, Fit time=0.050 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=360.309, BIC=371.406, Fit time=0.099 seconds
Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 0); AIC=359.224, BIC=368.102, Fit time=0.075 seconds
Fit ARIMA: order=(1, 0, 2

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=351.277, BIC=360.096, Fit time=0.065 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=351.125, BIC=359.944, Fit time=0.056 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=353.655, BIC=364.678, Fit time=0.080 seconds
Total fit time: 0.292 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=369.504, BIC=373.913, Fit time=0.016 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=362.048, BIC=368.662, Fit time=0.020 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=350.261, BIC=356.875, Fit time=0.040 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=369.504, BIC=373.913, Fit time=0.016 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=351.277, BIC=360.096, Fit time=0.064 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=351.125, BIC=359.944, Fit time=0.055 seconds
Fit ARIMA: order=(1, 1, 2

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=352.616, BIC=363.640, Fit time=0.089 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=352.486, BIC=363.509, Fit time=0.063 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=354.786, BIC=368.014, Fit time=0.104 seconds
Total fit time: 0.394 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=371.466, BIC=378.080, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=363.991, BIC=372.810, Fit time=0.051 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=351.810, BIC=360.629, Fit time=0.063 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 0); AIC=371.466, BIC=378.080, Fit time=0.014 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 0); AIC=352.616, BIC=363.640, Fit time=0.089 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 0, 0, 0); AIC=352.486, BIC=363.509, Fit time=0.122 seconds
Fit ARIMA: order=(1, 1, 2